# BILLE SUR GLISSIÈRE. EXPERIMENT

In [ ]:
import urllib
import os
from notebook import notebookapp

import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import math

import io
import base64
from ipywidgets import interact, interactive, fixed, interact_manual
import ipywidgets as widgets
from IPython.display import HTML

from bokeh.plotting import figure, curdoc
from bokeh.models.widgets import Slider, CheckboxButtonGroup, PreText
from bokeh.layouts import row, column, widgetbox
from bokeh.models import ColumnDataSource, Slider, Button, TextInput, Arrow, OpenHead, NormalHead, VeeHead
from bokeh.plotting import figure, show, ColumnDataSource

from bokeh.io import output_notebook, show, export_png

output_notebook()


### Import data from analysed video and video

In [ ]:

from IPython.display import HTML

### VIDEO 1

In [ ]:
video = io.open('data_bille/h1-dots.mov', 'r+b').read()
encoded = base64.b64encode(video)
HTML(data='''<video alt="test" controls>
                <source src="data:video/mp4;base64,{0}" type="video/mp4" />
             </video>'''.format(encoded.decode('ascii')))

### VIDEO 2

In [ ]:
video = io.open('data_bille/h3-dots.mov', 'r+b').read()
encoded = base64.b64encode(video)
HTML(data='''<video alt="test" controls>
                <source src="data:video/mp4;base64,{0}" type="video/mp4" />
             </video>'''.format(encoded.decode('ascii')))

### VIDEO 3

In [ ]:
video = io.open('data_bille/h5-dots.mov', 'r+b').read()
encoded = base64.b64encode(video)
HTML(data='''<video alt="test" controls>
                <source src="data:video/mp4;base64,{0}" type="video/mp4" />
             </video>'''.format(encoded.decode('ascii')))

# Bille sur glissière

On lâche la bille d'une hauteur h et on observe son comportement au point P en supposant qu'elle suit la trajectoire imposée par la glissière. 

La conservation de l'énergie donne: 
$$mgh = mgy_{p} + \frac{1}{2}mv_{p}^{2}$$

$$y_p = R ( 1 + \sin\theta)$$

$$v_p = \sqrt{2g[h - R ( 1 + \sin\theta)]}$$


La direction du vecteur vitesse en P s'obtient par le fait qu'il est tangent à la glissière

$$\vec v=\begin{pmatrix}-v_p\sin\theta\\v_p\cos\theta \end{pmatrix}$$


Cela permet de calculer la trajectoire parabolique qu'aurait une bille libre dans le champ de pesanteur lancée en P avec $\vec v$.

Un calcul plus juste tient compte de l'énergie de rotation de la bille. Dans ce cas la conservation de l'énergie devient: 
$$mgh = mgy_p + \frac{1}{2}mv_{p}^{2} + \frac{1}{2}Iw_{p}^{2}$$

$$I = \frac{2}{5}mr^2$$

$$w_p = \frac{v_p}{r}$$  avec r le rayon de la bille 


$$mgh = mgy_p + \frac{1}{2}mv_{p}^{2} + \frac{1}{2}\frac{2}{5}mr^2\frac{v_p^2}{r^2}$$

$$gh = gR(1 + \sin\theta) + \frac{7}{10}v_p^2$$

$$v_p = \sqrt{\frac{10}{7}g[h - R(1 + \sin\theta)]}$$

$\theta_p$ en tenant compte du moment d'inertie est donc $\sqrt{\frac{5}{7}} = 0,845$ fois le $v_p$ obtenu sans tenir en compte

In [ ]:
# Définir image de fons
img_path = 'data_bille/background.png'

# Importer des données du video
df = pd.read_csv('data_bille/data-05-04-19.csv', index_col=0)
df = df.rename({'VideoAnalysis: X (cm)': 'x', 'VideoAnalysis: Y (cm)':'y',
       'VideoAnalysis: X Velocity (cm/s)':'v_x', 'VideoAnalysis: Y Velocity (cm/s)':'v_y'}, axis='columns')

# Calculer les limites  
y_min = df['y'].min()
x_min = df['x'].min()
x_max = df['x'].max()
x_lowest = df.loc[df['y'] == y_min]['x'].iloc[0]

# Calculer le rayon de la boucle
df_h = df.loc[df['x'] < 20]
df_h = df_h.loc[df_h['x'] > -20]
R = df_h['y'].max() - df_h['y'].min()
idx = df_h.loc[df_h['y'] == df_h['y'].min()]
R = R/2

In [ ]:
# Définir parametres
g = 9.81
h = 2*R
omega = 20
to_rad = lambda x: np.pi*x/180.0
omegarad = to_rad(omega)

t = np.linspace(0, 2, 100)
neg_t = np.linspace(0, -1, 100)

# Eq. parametrique du parabole
x_p = lambda v, x, t: -v*np.sin(x)*t + R*np.cos(x)
y_p = lambda v, x, t: -0.5*g*t*t + v*np.cos(x)*t + R*(1 + np.sin(x))

# Eq. parametrique vitesse
vx = lambda v, ang: np.linspace(x_p(v, ang, t)[0], x_p(v, ang, t)[0] - 0.5*v*np.sin(ang), 10) 
vy = lambda v, ang: np.linspace(y_p(v, ang, t)[0], y_p(v, ang, t)[0] + 0.5*v*np.cos(ang), 10) 

# Define equations
vitesse_p = lambda h, omega_rad: math.sqrt(2*(g*h - g*R*(1 + np.sin(omega_rad)))) #Vitesse au point A
normale_p = lambda h, omega_rad: (1/R) * vitesse_p(h, omega_rad)**2 - 9.8*np.sin(omegarad) #Force normale au point A



In [ ]:
## PLOT

def modify_doc(doc):
    
    # Define data to plot
    x = -df['x']
    y = df['y'] - y_min
    x0 = x.iloc[0]
    y0 = y.iloc[0]

    # Define track
    slope2 = (y.iloc[6] - y.iloc[5]) / (x.iloc[6] - x.iloc[5])
    get_x = lambda y_ini: x0 - (y0 - y_ini)*(1/slope2)

    line_ini_y = np.linspace(2, y.iloc[0], 20)
    line_ini_x = get_x(line_ini_y)

    slope = np.arctan((y.iloc[6] - y.iloc[5]) / (x.iloc[6] - x.iloc[5]))
    line_end_x = x.iloc[-1] - np.linspace(0, 40, 20)*np.cos(-slope-0.02)
    line_end_y = y.iloc[-1] - np.linspace(0, 40, 20)*np.sin(-slope -0.02)

    angle = np.linspace(0, 2*np.pi, 100)
    
    # Conditions initiales 
    g = 9.81
    h = 2*R
    omega = 45
    omegarad = to_rad(omega)
    t = np.linspace(0, 2, 100)
    neg_t = np.linspace(0, -1, 100)
    vit_P = vitesse_p(h,omegarad)


    def initialize(h_init, omegarad):
        
        vit_P = vitesse_p(h_init,omegarad)
        n_P = normale_p(h_init,omegarad)
        x_parab = x_p(vit_P, omegarad, t)
        y_parab = y_p(vit_P, omegarad, t)
        
        x_parab_neg = x_p(vit_P, omegarad, neg_t)
        y_parab_neg = y_p(vit_P, omegarad, neg_t)
        
        norm_end_x = x_parab[0] - n_P*np.cos(omegarad)
        norm_end_y =  y_parab[0] - n_P*np.sin(omegarad)
              
        v_x, v_y = vx(vit_P, omegarad), vy(vit_P, omegarad)
        
        P = {
            'x_init': [get_x(h_init)], 'y_init':[h_init],
            'x_point_P': [x_parab[0]], 'y_point_P': [y_parab[0]]
        }
        
        CURV = {
            'x_point_P_parab_pos': x_parab, 'y_point_P_parab_pos': y_parab,
            'x_point_P_parab_neg': x_parab_neg, 'y_point_P_parab_neg': y_parab_neg
        }
        
        ANG = {
            'l_hor_x':np.linspace(0,R, 100), 'l_hor_y':[R]*100, 
            'l_ver_x': np.linspace(0, x_parab[0], 100), 'l_ver_y': np.linspace(R,y_parab[0],100)
            
        }
        
        FORCES = {
            'grav_x': [x_parab[0]]*10, 'grav_y': np.linspace(y_parab[0], -9.8+y_parab[0], 10),
            'arrow_up_grav_x': np.linspace(x_parab[0], 0.5 + x_parab[0], 10), 
            'arrow_up_grav_y': np.linspace(-9.8 + y_parab[0], 0.6 -9.8 + y_parab[0], 10),
            'arrow_down_grav_x': np.linspace(x_parab[0], -0.5 + x_parab[0], 10), 
            'arrow_down_grav_y': np.linspace(-9.8 + y_parab[0], 0.6 -9.8 + y_parab[0], 10),
            'norm_x': np.linspace(x_parab[0], norm_end_x, 10), 'norm_y': np.linspace(y_parab[0], norm_end_y, 10),
            'arrow_up_norm_x':  norm_end_x  + np.linspace(0, 0.1*n_P*np.cos(omegarad - np.pi*20/180), 10), 
            'arrow_up_norm_y':  norm_end_y  + np.linspace(0, 0.1*n_P*np.sin(omegarad - np.pi*20/180), 10), 
            'arrow_down_norm_x': norm_end_x + np.linspace(0, 0.1*n_P*np.cos(omegarad + np.pi*20/180), 10), 
            'arrow_down_norm_y': norm_end_y +  np.linspace(0, 0.1*n_P*np.sin(omegarad + np.pi*20/180), 10)
        }
              
        VEL = {
            'v_x': v_x,
            'v_y': v_y, 
            'arrow_up_vel_x': v_x[-1] + np.linspace(0, 0.1*vit_P*np.cos(omegarad - np.pi*70/180), 10),
            'arrow_up_vel_y': v_y[-1] + np.linspace(0, 0.1*vit_P*np.sin(omegarad - np.pi*70/180), 10), 
            'arrow_down_vel_x': v_x[-1] + np.linspace(0,- 0.1*vit_P*np.cos(omegarad + np.pi*70/180), 10),
            'arrow_down_vel_y': v_y[-1] + np.linspace(0,- 0.1*vit_P*np.sin(omegarad + np.pi*70/180), 10) 

        }
    
        return P, CURV, ANG, FORCES, VEL
    
  
    # Calculer les valeurs initiales
    P, C, ANG, FORCES, VEL = initialize(h, omegarad)

    # Créer dictionnaire pour acceder aux données
    source_P = ColumnDataSource(data = P)
    source_C = ColumnDataSource(data = C)
    source_ANGLE = ColumnDataSource(data = ANG)
    
    source_FORCES = ColumnDataSource(data = FORCES)
    source_VITESSE = ColumnDataSource(data= VEL)
    
    
    # Generate figure
    p = figure(title="Trajectory", plot_height=425, plot_width=950, x_range=(-57,53), y_range=(-5,45),\
               background_fill_color='#ffffff')
    # Set background
    p.image_url(url=[img_path], x=-56.5, y=45, w=110, h=45, alpha=0.2, angle=-np.pi*1/180)


    # Static part of the plot:
    # Plot track
    p.line(line_ini_x, line_ini_y, color='#635c74', alpha=0.7, line_width=1.5, legend='Track')
    #p.line(line_end_x, line_end_y, color='#635c74', alpha=0.7, line_width=1.5, legend='Track')
    p.line(R*np.sin(angle), R + R*np.cos(angle), color='#635c74', alpha=0.6, line_width=1, legend='Track')
    # Plot center of the loop
    p.circle(0,R,size=3, fill_color='#635c74', line_color='#635c74', legend='Center')


    # Variables:
    # Point initial
    p_ini = p.circle('x_init', 'y_init', source=source_P, size=6, fill_color='#e32020', line_color='#e32020', legend='Initial Point')

    # Point A
    p_a = p.circle('x_point_P','y_point_P', source=source_P, size=6, \
                   legend='A', line_color='#e32020', fill_color='#e32020')

    #Parabole
    parab = p.line('x_point_P_parab_pos', 'y_point_P_parab_pos', source=source_C, color='#e32020', line_width=1.5, alpha=0.8, \
                   legend='Parabola')

    parab_neg = p.line('x_point_P_parab_neg', 'y_point_P_parab_neg', source=source_C, color='#e32020', line_width=1.5, alpha=0.8, \
                   legend='Parabola')
    

    #Angle omega
    l_h = p.line('l_hor_x', 'l_hor_y', source=source_ANGLE, line_width=1, color='#000000', alpha=0.2)
    l_v = p.line('l_ver_x', 'l_ver_y', source=source_ANGLE, line_width=1 ,color='#000000', alpha=0.2)

    
    f_g = p.line('grav_x', 'grav_y', source=source_FORCES, line_width=1, color='#178717', alpha=0.8, legend='mg')
    a_u_f = p.line('arrow_up_grav_x', 'arrow_up_grav_y', source=source_FORCES, line_width=1, color='#178717', alpha=0.8, legend='mg')
    a_d_f = p.line('arrow_down_grav_x', 'arrow_down_grav_y', source=source_FORCES, line_width=1, color='#178717', alpha=0.8, legend='mg')


    f_n = p.line('norm_x', 'norm_y', source=source_FORCES, line_width=1.5, color='#064d06', alpha=0.8, legend='N')
    a_u_n = p.line('arrow_up_norm_x', 'arrow_up_norm_y', source=source_FORCES, line_width=1.5, color='#064d06', alpha=0.8, legend='N')

    a_d_n = p.line('arrow_down_norm_x', 'arrow_down_norm_y', source=source_FORCES, line_width=1.5, color='#064d06', alpha=0.8, legend='N')

    # Vitesse
    v = p.line('v_x', 'v_y', source=source_VITESSE, color='#01B9FF', legend='Velocity') 
    a_u = p.line('arrow_up_vel_x','arrow_up_vel_y', source=source_VITESSE, line_width=2)
    a_d = p.line('arrow_down_vel_x','arrow_down_vel_y', source=source_VITESSE, line_width=2)
    
    
    # Définir sliders 
    slider_h = Slider(start=2, end=3, step=0.05, value=2, title='Hauteur initiale')
    slider_om = Slider(start=0, end=90, step=1, value=45, title='Omega')
    
    pre = PreText(text='Point initiale.\tx:{:3.2f} \ty:{:3.2f} \nPoint A.\tx: {:3.2f} \ty: {:3.2f} \nVitesse.\t{:3.2f}[cm/s]\tx:{:3.2f}[cm/s]\ty:{:3.2f}[cm/s]'\
                  .format(source_P.data['x_init'][0], source_P.data['y_init'][0],source_P.data['x_point_P'][0], source_P.data['y_point_P'][0], \
                         vit_P, vit_P*np.cos(omegarad), vit_P*np.cos(omegarad)),
    width=500, height=100)
    
    
    def refresh_source(attrname, old, new):
        
        om = slider_om.value
        h = slider_h.value
        h = h*R
        
        omrad = to_rad(om)

        P, C, ANG, FORCES, VEL = initialize(h, omrad)
        
        # Créer dictionnaire pour acceder aux données
        source_P.data = P
        source_C.data = C
        source_ANGLE.data = ANG

        source_FORCES.data = FORCES
        source_VITESSE.data = VEL
        
        vit_P = vitesse_p(h, omrad)
        x_p = source_C.data['x_point_P_parab_pos'][0]
        y_p = source_C.data['y_point_P_parab_pos'][0]
        
        pre.text='Point initiale.\tx:{:3.2f} \ty:{:3.2f} \nPoint A.\tx: {:3.2f} \ty: {:3.2f} \nVitesse.\t{:3.2f}[cm/s]\tx:{:3.2f}[cm/s]\ty:{:3.2f}[cm/s]'\
                  .format(source_P.data['x_init'][0], source_P.data['y_init'][0],source_P.data['x_point_P'][0], source_P.data['y_point_P'][0], \
                         vit_P, vit_P*np.cos(omegarad), vit_P*np.cos(omegarad))
        
        
    slider_h.on_change('value', refresh_source)
    slider_om.on_change('value', refresh_source)
    
    layout = column(
        row(p),
        row(slider_h, slider_om),
        row(pre)
    )

    
    doc.add_root(layout)
    

In [ ]:
def remote_jupyter_proxy_url(port):
    """
    Callable to configure Bokeh's show method when a proxy must be
    configured.

    If port is None we're asking about the URL
    for the origin header.
    """
    
    base_url = os.environ['EXTERNAL_URL']
    host = urllib.parse.urlparse(base_url).netloc

    # If port is None we're asking for the URL origin
    # so return the public hostname.
    if port is None:
        return host

    service_url_path = os.environ['JUPYTERHUB_SERVICE_PREFIX']
    proxy_url_path = 'proxy/%d' % port

    user_url = urllib.parse.urljoin(base_url, service_url_path)
    full_url = urllib.parse.urljoin(user_url, proxy_url_path)
    return full_url



def show_document(doc):
    servers = list(notebookapp.list_running_servers())[0]
    if servers['hostname'] == 'localhost':
        show(doc) 
    else:
        show(doc, notebook_url=remote_jupyter_proxy_url)


        
        
show_document(modify_doc) 